In [6]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import gc

In [17]:
# read_file

path = 'dataset/'
start_time = time.time()
train = pd.read_csv(path+"train.csv", skiprows=160000000, nrows=1000000)
test = pd.read_csv(path+"test.csv")
train.columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time', 'is_attributed']
print('[{}] Finished loading data'.format(time.time() - start_time))
train.head()

[48.8827168941] Finished loading data


,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,11846,12,1,13,259,2017-11-09 08:17:26,NaN,0
1,5147,19,0,0,347,2017-11-09 08:17:26,NaN,0
2,11782,9,1,8,127,2017-11-09 08:17:26,NaN,0
3,33867,35,1,19,21,2017-11-09 08:17:26,2017-11-09 09:05:37,1
4,110589,3,1,23,280,2017-11-09 08:17:26,NaN,0


In [18]:
def dataPreProcessTime(df):
    df['datetime'] = pd.to_datetime(df['click_time'])
    #df['dow']      = df['datetime'].dt.dayofweek.astype(int)
    #df['month']    = df['datetime'].dt.month.astype(int)
    df['hour']     = df['datetime'].dt.hour.astype('uint8')
    df['day']      = df['datetime'].dt.day.astype('uint8')
    #df.drop(['click_time','datetime'], axis=1, inplace=True)
    return df

# the single character 
def feature_ipcount(df):
    print("feature: ipcount ")
    ip_count = df.groupby(['ip'])['os'].count().reset_index()
    ip_count.columns = ['ip', 'ipcount']
    df = pd.merge(df, ip_count, on='ip', how='left', sort=False)
    del ip_count
    gc.collect()
    return df

def feature_appcount(df):
    print("feature: appcount")
    app_count = df.groupby(['app'])['os'].count().reset_index()
    app_count.columns = ['app','appcount']
    df = pd.merge(df, app_count, on='app', how='left', sort=False)
    del app_count
    gc.collect()
    return df

def feature_channelcount(df):
    print("feature: channelcount ")
    channel_count = df.groupby(['channel'])['os'].count().reset_index()
    channel_count.columns = ['channel', 'channelcount']
    df = pd.merge(df, channel_count, on='channel', how='left', sort=False)
    del channel_count
    gc.collect()
    return df

def feature_devicecount(df):
    print("feature: devicecount ")
    device_count = df.groupby(['device'])['os'].count().reset_index()
    device_count.columns = ['device', 'devicecount']
    df = pd.merge(df, device_count, on='device', how='left', sort=False)
    del device_count
    gc.collect()
    return df

## two combined character

# compute the realtion between the time and charcter 

def feature_iphour(df):
    print("feature ip_hour(ip+time feature)")
    iphour = df[['ip','hour','channel']].groupby(by=['ip','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_hour'})
    df = df.merge(iphour, on=['ip','hour'], how='left')
    del iphour
    gc.collect()
    return df

def feature_devicehour(df):
    print("feature device_hour")
    device_hour = df[['device','hour','channel']].groupby(by=['device','hour'])[['channel']].count().reset_index().rename(index=str, columns = {'channel':'device_hour'})
    df = df.merge(device_hour, on=['device','hour'], how = 'left')
    del device_hour
    gc.collect()
    return df
    
def feature_apphour(df):
    print("feature app_hour")
    app_hour = df[['app','hour','channel']].groupby(by=['app','hour'])[['channel']].count().reset_index().rename(index=str, columns = {'channel':'app_hour'})
    df= df.merge(app_hour, on=['app','hour'], how='left')
    del app_hour
    gc.collect()
    return df

# compute the relation between the two character

def feature_ipapp(df):
    print("feature ip_app ")
    ip_app = df[['ip', 'app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app'})
    df = df.merge(ip_app, on=['ip','app'], how='left')
    del ip_app
    gc.collect()
    return df

def feature_ipchan(df):
    print("feature ip_chan")
    ip_channel = df[['ip', 'channel', 'app']].groupby(by=['ip', 'channel'])[['app']].count().reset_index().rename(index=str, columns={'app': 'ip_channel'})
    df = df.merge(ip_channel, on=['ip','channel'], how='left')
    del ip_channel
    gc.collect()
    return df

def feature_appchan(df):
    print("feature app_channel ")
    ip_app = df[['os', 'app', 'channel']].groupby(by=['channel', 'app'])[['os']].count().reset_index().rename(index=str, columns={'os': 'app_chan'})
    df = df.merge(ip_app, on=['channel','app'], how='left')
    del ip_app
    gc.collect()
    return df

def feature_deviceos(df):
    print("feature device_os ")
    device_os = df[['os', 'device', 'channel']].groupby(by=['device', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'device_os'})
    df = df.merge(device_os, on=['os','device'], how='left')
    del device_os
    gc.collect()
    return df

## three combined character is not fine 

#def feature_ipappos(df):
#    print("feature ip_app_os")
#    ipappos = df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os'})
#    df = df.merge(ipappos, on=['ip','app', 'os'], how='left')
#    del ipappos
#    gc.collect()
#    return df

#def feature_ipappchan(df):
#    print("feature ip_app_chan")
#    ipappchan = df[['ip','app', 'channel', 'os']].groupby(by=['ip', 'app', 'channel'])[['os']].count().reset_index().rename(index=str, columns={'os': 'ip_app_channel'})
#    df = df.merge(ipappchan, on=['ip','app', 'channel'], how='left')
#    del ipappchan
#    gc.collect()
#    return df 



# Adding features with var and mean hour (inspired from nuhsikander's script)

# this character is not right 
#def feature_ipdaychan_var_hour(df):
#    print('grouping by : ip_day_chl_var_hour')
#    ipdaychan = df[['ip','day','hour','channel']].groupby(by=['ip','day','channel'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_day_chan_var_hour'})
#    df = df.merge(ipdaychan, on=['ip','day','channel'], how='left')
#    del ipdaychan
#    gc.collect()
#    return df

def feature_ipappos_var_hour(df):
    print('grouping by : ip_app_os_var_hour')
    gp = df[['ip','app', 'os', 'hour']].groupby(by=['ip', 'app', 'os'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_app_os_var_hour'})
    df = df.merge(gp, on=['ip','app', 'os'], how='left')
    del gp
    gc.collect()
    return df

def feature_ipappchan_var_day(df):
    print('grouping by : ip_app_channel_var_day')
    gp = df[['ip','app', 'channel', 'day']].groupby(by=['ip', 'app', 'channel'])[['day']].var().reset_index().rename(index=str, columns={'day': 'ip_app_channel_var_day'})
    df = df.merge(gp, on=['ip','app', 'channel'], how='left')
    del gp 
    gc.collect()
    return df

def feature_ip_app_chl_mean_hour(df):
    print('grouping by : ip_app_chl_mean_hour')
    gp = df[['ip','app', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].mean().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_mean_hour'})
    df = df.merge(gp, on=['ip','app', 'channel'], how='left')
    del gp
    gc.collect()
    return df

def merge_feature(df):
    train = dataPreProcessTime(df)
    
    # single feature
    train = feature_ipcount(train)
    train = feature_appcount(train)
    train = feature_channelcount(train)
    train = feature_devicecount(train)
    
    #combine time character
    train = feature_iphour(train)
    train = feature_apphour(train)
    train = feature_devicehour(train)
    
    # combine two character
    train = feature_ipapp(train)
    train = feature_ipchan(train)
    train = feature_appchan(train)
    train = feature_deviceos(train)
    
    # combine the time feature
    #train = feature_ipdaychan_var_hour(train)
    #train = feature_ipappos_var_hour(train)
    #train = feature_ipappchan_var_day(train)
    #train = feature_ip_app_chl_mean_hour(train)
    del df
    gc.collect()
    return train

train=merge_feature(train)
train.head()

feature: ipcount 
feature: appcount
feature: channelcount 
feature: devicecount 
feature ip_hour(ip+time feature)
feature app_hour
feature device_hour
feature ip_app 
feature ip_chan
feature app_channel 
feature device_os 


,ip,app,device,os,channel,click_time,attributed_time,is_attributed,datetime,hour,...,appcount,channelcount,devicecount,ip_hour,app_hour,device_hour,ip_app,ip_channel,app_chan,device_os
0,11846,12,1,13,259,2017-11-09 08:17:26,NaN,0,2017-11-09 08:17:26,8,...,113789,26937,944293,3,113789,944293,1,1,16721,201090
1,5147,19,0,0,347,2017-11-09 08:17:26,NaN,0,2017-11-09 08:17:26,8,...,13028,10113,12563,2395,13028,12563,91,78,6656,4010
2,11782,9,1,8,127,2017-11-09 08:17:26,NaN,0,2017-11-09 08:17:26,8,...,123313,22123,944293,19,123313,944293,5,3,22123,27122
3,33867,35,1,19,21,2017-11-09 08:17:26,2017-11-09 09:05:37,1,2017-11-09 08:17:26,8,...,226,1636,944293,16,226,944293,1,1,131,232033
4,110589,3,1,23,280,2017-11-09 08:17:26,NaN,0,2017-11-09 08:17:26,8,...,124663,9024,944293,231,124663,944293,38,25,8996,10599


In [19]:
train_y = train['is_attributed']
# add inplace=True 直接替换
train_x = train.drop(['is_attributed', 'attributed_time','ip','click_time','datetime'], axis=1)
train_x.head()

,app,device,os,channel,hour,day,ipcount,appcount,channelcount,devicecount,ip_hour,app_hour,device_hour,ip_app,ip_channel,app_chan,device_os
0,12,1,13,259,8,9,3,113789,26937,944293,3,113789,944293,1,1,16721,201090
1,19,0,0,347,8,9,2395,13028,10113,12563,2395,13028,12563,91,78,6656,4010
2,9,1,8,127,8,9,19,123313,22123,944293,19,123313,944293,5,3,22123,27122
3,35,1,19,21,8,9,16,226,1636,944293,16,226,944293,1,1,131,232033
4,3,1,23,280,8,9,231,124663,9024,944293,231,124663,944293,38,25,8996,10599


In [20]:
params = {'eta': 0.1, 
          'max_depth': 4, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':100,
          'alpha':4,
          'objective': 'binary:logistic', 
          'eval_metric': 'auc', 
          'random_state': 99, 
          'scale_pos_weight': 150,
          'silent': True}

In [21]:
del train
gc.collect()

323

In [22]:
from sklearn.model_selection import KFold
num_folds = 3
seed = 7 
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

valid_result = []
for train_index, test_index in kf.split(train_x):
    evals_result = {}
    watchlist = [(xgb.DMatrix(train_x.ix[train_index], train_y.ix[train_index]), 'train'), (xgb.DMatrix(train_x.ix[test_index], train_y.ix[test_index]), 'valid')]
    dtrain = xgb.DMatrix(train_x.ix[train_index], train_y.ix[train_index])
    model = xgb.train(params, dtrain, 50 , watchlist, maximize=True, verbose_eval=10,evals_result=evals_result,early_stopping_rounds = 40)
    valid_result.append( evals_result['valid']['auc'][-1])
print('[{}] Finish XGBoost Training'.format(time.time() - start_time))

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


[0]	train-auc:0.923269	valid-auc:0.929891
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[10]	train-auc:0.95149	valid-auc:0.954198
[20]	train-auc:0.954543	valid-auc:0.956724
[30]	train-auc:0.958432	valid-auc:0.959113
[40]	train-auc:0.961788	valid-auc:0.960128
[49]	train-auc:0.964257	valid-auc:0.961865
[0]	train-auc:0.916817	valid-auc:0.907099
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[10]	train-auc:0.953119	valid-auc:0.945387
[20]	train-auc:0.956766	valid-auc:0.950233
[30]	train-auc:0.96041	valid-auc:0.954106
[40]	train-auc:0.963742	valid-auc:0.954596
[49]	train-auc:0.966545	valid-auc:0.955608
[0]	train-auc:0.915697	valid-auc:0.907443
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[10]	train-auc:0.

In [23]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
plot_importance(model)
plt.gcf().savefig('feature_importance_xgb.png')

In [24]:
fscore=pd.Series(model.get_fscore()).sort_values(ascending = False)
print fscore

channelcount    87
appcount        82
ipcount         75
app             72
channel         70
ip_hour         68
app_chan        59
device_os       47
app_hour        45
ip_app          32
os              25
ip_channel      25
device          21
devicecount     10
device_hour      5
dtype: int64


In [25]:
mean_result = reduce(lambda x, y: x + y, valid_result)/len(valid_result)
print mean_result

0.957198333333


In [26]:
sub = pd.DataFrame()
sub['click_id'] = test['click_id']
test.drop('click_id', axis=1, inplace=True)

print('[{}] Start XGBoost Test'.format(time.time() - start_time))

[463.359972954] Start XGBoost Test


In [27]:
# deal with the test data
test=merge_feature(test)
test.drop(['click_time','datetime','ip'], axis=1, inplace=True)
test.head()

feature: ipcount 
feature: appcount
feature: channelcount 
feature: devicecount 
feature ip_hour(ip+time feature)
feature app_hour
feature device_hour
feature ip_app 
feature ip_chan
feature app_channel 
feature device_os 


,app,device,os,channel,hour,day,ipcount,appcount,channelcount,devicecount,ip_hour,app_hour,device_hour,ip_app,ip_channel,app_chan,device_os
0,9,1,3,107,4,10,91,2872176,1214650,17360269,34,475418,3175341,28,4,269116,274027
1,9,1,3,466,4,10,2083,2872176,422089,17360269,403,475418,3175341,289,38,372406,274027
2,21,1,19,128,4,10,2135,704267,521176,17360269,229,77207,3175341,312,158,219966,4213376
3,15,1,13,111,4,10,1201,1079113,67038,17360269,239,209436,3175341,42,2,67038,3759710
4,12,1,13,328,4,10,208,2306083,191986,17360269,60,396028,3175341,24,2,191986,3759710


In [ ]:
sub['is_attributed'] = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
sub.to_csv('xgb_sub.csv',index=False)